# ALASKA2 Image Steganalysis
### Detecting Hidden Data in Images Using Deep Learning
CQ 2021/21  
Nguyễn Văn Hậu  - 21120449 
Nguyễn Đức Minh Quân - 20120357

**Project Objective:**
The project is based on the ALASKA2 Image Steganalysis competition on Kaggle, which requires teams to create algorithms capable of efficiently detecting hidden data.  
## 1. Problem Statement
### 1.1. Steganalysis Problem  
Steganography is a technique that hides information within digital media (images, audio, video) to conceal secret messages without significantly altering the original data. In contrast, steganalysis aims to detect and decode this hidden information.

In this problem, we will work with a dataset containing images from various sources (50 different cameras). Some of these images contain hidden messages, while others do not. The goal is to develop a model that can accurately identify which images contain hidden data.

### 1.2. Challenges
Steganographic techniques are advanced.  
Images come from different cameras, adding variability to the dataset.  
Finding the right features for detection is complex, as small pixel changes are hard to spot.  
### 1.3. Approach


| Week                | work                                                                 |
|---------------------|------------------------------------------------------------------------|
| Week 01 (17/3 - 23/3)  | Research steganography, collect papers, explore ALASKA2 dataset      |
| Week 02 - 03 (24/3 - 06/4) | Preprocess dataset, analyze image distributions, extract features |
| Week 04 - 05 (07/4 - 20/4) | Extract deep learning features, experiment with different preprocessing techniques |
| Week 06 - 07 (21/4 - 04/5) | Train models, optimize hyperparameters, evaluate different approaches |
| Week 08 (05/5 - 11/5)  | Submit initial results to Kaggle, evaluate performance              |
| Week 09 - 10 (12/5 - 25/5) | - Improve accuracy with ensemble learning <br> - Prepare presentation, submit final report |


## 2. Data exploration

In [9]:
import pandas as pd
BASE_PATH = "/alaska2-image-steganalysis"
train_imageids = pd.Series(os.listdir(BASE_PATH + '/Cover')).sort_values(ascending=True).reset_index(drop=True)
test_imageids = pd.Series(os.listdir(BASE_PATH + '/Test')).sort_values(ascending=True).reset_index(drop=True)

ModuleNotFoundError: No module named 'pandas'

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from tqdm.notebook import tqdm
import random
from PIL import Image
import PIL.ExifTags
from collections import Counter
import glob
import math
from scipy.stats import ks_2samp
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Define paths
KAGGLE_PATH = '/alaska2-image-steganalysis/'
TRAIN_PATH = os.path.join(KAGGLE_PATH, 'train')
TEST_PATH = os.path.join(KAGGLE_PATH, 'test')
SAMPLE_SUBMISSION = os.path.join(KAGGLE_PATH, 'sample_submission.csv')

# 1. Basic Dataset Information
print("=== ALASKA2 Image Steganalysis Dataset ===")

# Get list of all train images
all_train_imgs = glob.glob(os.path.join(TRAIN_PATH, "*.jpg"))
print(f"Total training images: {len(all_train_imgs)}")

# Identify cover images and stego categories
cover_imgs = [img for img in all_train_imgs if os.path.basename(img).startswith("Cover")]
jmipod_imgs = [img for img in all_train_imgs if os.path.basename(img).startswith("JMiPOD")]
juniward_imgs = [img for img in all_train_imgs if os.path.basename(img).startswith("JUNIWARD")]
uerd_imgs = [img for img in all_train_imgs if os.path.basename(img).startswith("UERD")]

print(f"Cover (clean) images: {len(cover_imgs)}")
print(f"JMiPOD stego images: {len(jmipod_imgs)}")
print(f"JUNIWARD stego images: {len(juniward_imgs)}")
print(f"UERD stego images: {len(uerd_imgs)}")

# Get list of test images
test_imgs = glob.glob(os.path.join(TEST_PATH, "*.jpg"))
print(f"Test images: {len(test_imgs)}")

# Class distribution
class_distribution = {
    'Cover': len(cover_imgs),
    'JMiPOD': len(jmipod_imgs),
    'JUNIWARD': len(juniward_imgs),
    'UERD': len(uerd_imgs)
}

plt.figure(figsize=(10, 6))
sns.barplot(x=list(class_distribution.keys()), y=list(class_distribution.values()))
plt.title('Class Distribution in Training Set')
plt.ylabel('Number of Images')
plt.xlabel('Image Type')
for i, v in enumerate(class_distribution.values()):
    plt.text(i, v + 100, str(v), ha='center')
plt.tight_layout()
plt.show()

# 2. Image Dimensions Analysis
print("\n=== Image Dimensions Analysis ===")

def get_image_dimensions(img_path):
    """Get image dimensions (width, height)"""
    img = Image.open(img_path)
    return img.size

# Sample a subset of images for dimension analysis
sample_size = min(1000, len(all_train_imgs))
sampled_imgs = random.sample(all_train_imgs, sample_size)

dimensions = [get_image_dimensions(img) for img in tqdm(sampled_imgs, desc="Getting image dimensions")]
widths, heights = zip(*dimensions)

print(f"Width statistics: Min={min(widths)}, Max={max(widths)}, Mean={np.mean(widths):.2f}, Median={np.median(widths)}")
print(f"Height statistics: Min={min(heights)}, Max={max(heights)}, Mean={np.mean(heights):.2f}, Median={np.median(heights)}")

# Plot dimension distribution
plt.figure(figsize=(15, 6))

plt.subplot(1, 2, 1)
plt.hist(widths, bins=20, alpha=0.7)
plt.title('Distribution of Image Widths')
plt.xlabel('Width (pixels)')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
plt.hist(heights, bins=20, alpha=0.7)
plt.title('Distribution of Image Heights')
plt.xlabel('Height (pixels)')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

# Scatter plot of width vs height
plt.figure(figsize=(10, 8))
plt.scatter(widths, heights, alpha=0.5)
plt.title('Image Dimensions: Width vs Height')
plt.xlabel('Width (pixels)')
plt.ylabel('Height (pixels)')
plt.grid(True, linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

# 3. Image Metadata Analysis
print("\n=== Image Metadata Analysis ===")

def extract_exif_data(img_path):
    """Extract EXIF data from an image"""
    try:
        img = Image.open(img_path)
        exif = {
            PIL.ExifTags.TAGS[k]: v
            for k, v in img._getexif().items()
            if k in PIL.ExifTags.TAGS
        } if img._getexif() else {}
        return exif
    except:
        return {}

# Sample a smaller subset for EXIF analysis (to save time)
exif_sample_size = min(200, len(all_train_imgs))
exif_sampled_imgs = random.sample(all_train_imgs, exif_sample_size)

# Collect camera models
camera_models = []
for img_path in tqdm(exif_sampled_imgs, desc="Extracting EXIF data"):
    exif = extract_exif_data(img_path)
    model = exif.get('Model', 'Unknown')
    camera_models.append(model)

# Count camera models
camera_counts = Counter(camera_models)
top_cameras = dict(camera_counts.most_common(10))

plt.figure(figsize=(12, 6))
sns.barplot(x=list(top_cameras.keys()), y=list(top_cameras.values()))
plt.title('Top 10 Camera Models in Dataset')
plt.xlabel('Camera Model')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# 4. Image Quality and Statistical Analysis
print("\n=== Image Quality and Statistical Analysis ===")

def compute_image_statistics(img_path):
    """Compute basic statistical properties of an image"""
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    
    # Basic statistics
    mean = np.mean(img)
    std = np.std(img)
    median = np.median(img)
    min_val = np.min(img)
    max_val = np.max(img)
    
    # Compute entropy (measure of randomness)
    histogram = cv2.calcHist([img], [0], None, [256], [0, 256])
    histogram = histogram / histogram.sum()
    entropy = -np.sum(histogram * np.log2(histogram + 1e-7))
    
    return {
        'mean': mean,
        'std': std,
        'median': median,
        'min': min_val,
        'max': max_val,
        'entropy': entropy
    }

# Sample images from each category
num_samples = min(100, len(cover_imgs))
sampled_cover = random.sample(cover_imgs, num_samples)
sampled_jmipod = random.sample(jmipod_imgs, num_samples)
sampled_juniward = random.sample(juniward_imgs, num_samples)
sampled_uerd = random.sample(uerd_imgs, num_samples)

# Compute statistics for each category
stats_cover = [compute_image_statistics(img) for img in tqdm(sampled_cover, desc="Cover stats")]
stats_cover = [s for s in stats_cover if s is not None]  # Remove None values

stats_jmipod = [compute_image_statistics(img) for img in tqdm(sampled_jmipod, desc="JMiPOD stats")]
stats_jmipod = [s for s in stats_jmipod if s is not None]

stats_juniward = [compute_image_statistics(img) for img in tqdm(sampled_juniward, desc="JUNIWARD stats")]
stats_juniward = [s for s in stats_juniward if s is not None]

stats_uerd = [compute_image_statistics(img) for img in tqdm(sampled_uerd, desc="UERD stats")]
stats_uerd = [s for s in stats_uerd if s is not None]

# Convert to DataFrame for easier analysis
df_cover = pd.DataFrame(stats_cover)
df_cover['type'] = 'Cover'

df_jmipod = pd.DataFrame(stats_jmipod)
df_jmipod['type'] = 'JMiPOD'

df_juniward = pd.DataFrame(stats_juniward)
df_juniward['type'] = 'JUNIWARD'

df_uerd = pd.DataFrame(stats_uerd)
df_uerd['type'] = 'UERD'

# Combine all stats
df_all_stats = pd.concat([df_cover, df_jmipod, df_juniward, df_uerd])

# Visualize statistics across categories
plt.figure(figsize=(20, 15))

metrics = ['mean', 'std', 'entropy', 'median', 'min', 'max']
for i, metric in enumerate(metrics):
    plt.subplot(3, 2, i+1)
    sns.boxplot(x='type', y=metric, data=df_all_stats)
    plt.title(f'Distribution of {metric.capitalize()} by Image Type')
    plt.xlabel('Image Type')
    plt.ylabel(metric.capitalize())

plt.tight_layout()
plt.show()

# Statistical tests to compare distributions
print("\n=== Statistical Tests: Cover vs Stego ===")
for metric in metrics:
    print(f"\nComparing {metric} distributions:")
    
    # KS test for Cover vs each stego method
    ks_jmipod = ks_2samp(df_cover[metric], df_jmipod[metric])
    ks_juniward = ks_2samp(df_cover[metric], df_juniward[metric])
    ks_uerd = ks_2samp(df_cover[metric], df_uerd[metric])
    
    print(f"Cover vs JMiPOD: KS statistic={ks_jmipod.statistic:.4f}, p-value={ks_jmipod.pvalue:.6f}")
    print(f"Cover vs JUNIWARD: KS statistic={ks_juniward.statistic:.4f}, p-value={ks_juniward.pvalue:.6f}")
    print(f"Cover vs UERD: KS statistic={ks_uerd.statistic:.4f}, p-value={ks_uerd.pvalue:.6f}")

# 5. Pixel Value Distribution Analysis
print("\n=== Pixel Value Distribution Analysis ===")

def compute_histogram(img_path):
    """Compute histogram of pixel values"""
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    
    hist = cv2.calcHist([img], [0], None, [256], [0, 256])
    return hist.flatten()

# Sample smaller set for histogram analysis
hist_sample_size = 20
hist_cover = [compute_histogram(img) for img in random.sample(cover_imgs, hist_sample_size)]
hist_cover = [h for h in hist_cover if h is not None]

hist_jmipod = [compute_histogram(img) for img in random.sample(jmipod_imgs, hist_sample_size)]
hist_jmipod = [h for h in hist_jmipod if h is not None]

hist_juniward = [compute_histogram(img) for img in random.sample(juniward_imgs, hist_sample_size)]
hist_juniward = [h for h in hist_juniward if h is not None]

hist_uerd = [compute_histogram(img) for img in random.sample(uerd_imgs, hist_sample_size)]
hist_uerd = [h for h in hist_uerd if h is not None]

# Average histograms
avg_hist_cover = np.mean(hist_cover, axis=0)
avg_hist_jmipod = np.mean(hist_jmipod, axis=0)
avg_hist_juniward = np.mean(hist_juniward, axis=0)
avg_hist_uerd = np.mean(hist_uerd, axis=0)

# Plot average histograms
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.plot(avg_hist_cover, color='blue')
plt.title('Average Histogram: Cover Images')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.subplot(2, 2, 2)
plt.plot(avg_hist_jmipod, color='red')
plt.title('Average Histogram: JMiPOD Images')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.subplot(2, 2, 3)
plt.plot(avg_hist_juniward, color='green')
plt.title('Average Histogram: JUNIWARD Images')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.subplot(2, 2, 4)
plt.plot(avg_hist_uerd, color='purple')
plt.title('Average Histogram: UERD Images')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

# Plot histogram differences
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(avg_hist_jmipod - avg_hist_cover)
plt.title('Histogram Difference: JMiPOD - Cover')
plt.xlabel('Pixel Value')
plt.ylabel('Difference')
plt.grid(True, linestyle='--', alpha=0.7)

plt.subplot(1, 3, 2)
plt.plot(avg_hist_juniward - avg_hist_cover)
plt.title('Histogram Difference: JUNIWARD - Cover')
plt.xlabel('Pixel Value')
plt.ylabel('Difference')
plt.grid(True, linestyle='--', alpha=0.7)

plt.subplot(1, 3, 3)
plt.plot(avg_hist_uerd - avg_hist_cover)
plt.title('Histogram Difference: UERD - Cover')
plt.xlabel('Pixel Value')
plt.ylabel('Difference')
plt.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# 6. Noise Analysis
print("\n=== Noise Residual Analysis ===")

def compute_noise_residual(img_path):
    """Compute noise residual from an image using a denoising filter"""
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    
    # Apply denoising filter
    denoised = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)
    
    # Calculate residual (difference between original and denoised)
    residual = img.astype(np.float32) - denoised.astype(np.float32)
    
    # Compute statistics on the residual
    mean = np.mean(residual)
    std = np.std(residual)
    entropy = -np.sum(np.histogram(residual, bins=100, density=True)[0] * 
                       np.log2(np.histogram(residual, bins=100, density=True)[0] + 1e-10))
    
    return {
        'mean': mean,
        'std': std,
        'entropy': entropy
    }

# Sample images for noise analysis
noise_sample_size = min(50, len(cover_imgs))
sampled_cover_noise = random.sample(cover_imgs, noise_sample_size)
sampled_jmipod_noise = random.sample(jmipod_imgs, noise_sample_size)
sampled_juniward_noise = random.sample(juniward_imgs, noise_sample_size)
sampled_uerd_noise = random.sample(uerd_imgs, noise_sample_size)

# Compute noise residuals
noise_cover = [compute_noise_residual(img) for img in tqdm(sampled_cover_noise, desc="Cover noise")]
noise_cover = [n for n in noise_cover if n is not None]

noise_jmipod = [compute_noise_residual(img) for img in tqdm(sampled_jmipod_noise, desc="JMiPOD noise")]
noise_jmipod = [n for n in noise_jmipod if n is not None]

noise_juniward = [compute_noise_residual(img) for img in tqdm(sampled_juniward_noise, desc="JUNIWARD noise")]
noise_juniward = [n for n in noise_juniward if n is not None]

noise_uerd = [compute_noise_residual(img) for img in tqdm(sampled_uerd_noise, desc="UERD noise")]
noise_uerd = [n for n in noise_uerd if n is not None]

# Convert to DataFrame
df_noise_cover = pd.DataFrame(noise_cover)
df_noise_cover['type'] = 'Cover'

df_noise_jmipod = pd.DataFrame(noise_jmipod)
df_noise_jmipod['type'] = 'JMiPOD'

df_noise_juniward = pd.DataFrame(noise_juniward)
df_noise_juniward['type'] = 'JUNIWARD'

df_noise_uerd = pd.DataFrame(noise_uerd)
df_noise_uerd['type'] = 'UERD'

# Combine all noise stats
df_all_noise = pd.concat([df_noise_cover, df_noise_jmipod, df_noise_juniward, df_noise_uerd])

# Visualize noise statistics
plt.figure(figsize=(15, 5))

noise_metrics = ['mean', 'std', 'entropy']
for i, metric in enumerate(noise_metrics):
    plt.subplot(1, 3, i+1)
    sns.boxplot(x='type', y=metric, data=df_all_noise)
    plt.title(f'Noise Residual {metric.capitalize()} by Image Type')
    plt.xlabel('Image Type')
    plt.ylabel(metric.capitalize())

plt.tight_layout()
plt.show()

# 7. Visual Comparison of Cover and Stego Pairs
print("\n=== Visual Comparison of Cover-Stego Pairs ===")

def visualize_stego_differences(cover_path, stego_path, amplification=10):
    """Visualize differences between cover and stego image pairs"""
    cover_name = os.path.basename(cover_path)
    stego_name = os.path.basename(stego_path)
    
    cover = cv2.imread(cover_path)
    stego = cv2.imread(stego_path)
    
    if cover is None or stego is None:
        return None
    
    # Calculate difference and amplify it for visualization
    diff = cv2.absdiff(cover, stego)
    diff_amplified = cv2.convertScaleAbs(diff, alpha=amplification)
    
    # Calculate noise residuals
    cover_gray = cv2.cvtColor(cover, cv2.COLOR_BGR2GRAY)
    stego_gray = cv2.cvtColor(stego, cv2.COLOR_BGR2GRAY)
    
    cover_denoised = cv2.fastNlMeansDenoising(cover_gray, None, 10, 7, 21)
    stego_denoised = cv2.fastNlMeansDenoising(stego_gray, None, 10, 7, 21)
    
    cover_residual = cover_gray - cover_denoised
    stego_residual = stego_gray - stego_denoised
    
    residual_diff = cv2.absdiff(stego_residual, cover_residual)
    residual_diff_amplified = cv2.convertScaleAbs(residual_diff, alpha=amplification)
    
    return {
        'cover': cover,
        'stego': stego,
        'diff': diff,
        'diff_amplified': diff_amplified,
        'cover_residual': cover_residual,
        'stego_residual': stego_residual,
        'residual_diff': residual_diff,
        'residual_diff_amplified': residual_diff_amplified,
        'cover_name': cover_name,
        'stego_name': stego_name
    }

# Select sample pairs
num_pairs = 3
cover_samples = random.sample(cover_imgs, num_pairs)

# Create pairs
jmipod_pairs = [(cover, cover.replace('Cover', 'JMiPOD')) for cover in cover_samples]
juniward_pairs = [(cover, cover.replace('Cover', 'JUNIWARD')) for cover in cover_samples]
uerd_pairs = [(cover, cover.replace('Cover', 'UERD')) for cover in cover_samples]

# Visualize JMiPOD pairs
for cover_path, stego_path in jmipod_pairs:
    result = visualize_stego_differences(cover_path, stego_path)
    if result is None:
        continue
        
    plt.figure(figsize=(20, 10))
    
    plt.subplot(2, 4, 1)
    plt.imshow(cv2.cvtColor(result['cover'], cv2.COLOR_BGR2RGB))
    plt.title(f'Cover Image: {result["cover_name"]}')
    plt.axis('off')
    
    plt.subplot(2, 4, 2)
    plt.imshow(cv2.cvtColor(result['stego'], cv2.COLOR_BGR2RGB))
    plt.title(f'JMiPOD Stego: {result["stego_name"]}')
    plt.axis('off')
    
    plt.subplot(2, 4, 3)
    plt.imshow(cv2.cvtColor(result['diff'], cv2.COLOR_BGR2RGB))
    plt.title('Difference (Unscaled)')
    plt.axis('off')
    
    plt.subplot(2, 4, 4)
    plt.imshow(cv2.cvtColor(result['diff_amplified'], cv2.COLOR_BGR2RGB))
    plt.title('Difference (Amplified 10x)')
    plt.axis('off')
    
    plt.subplot(2, 4, 5)
    plt.imshow(result['cover_residual'], cmap='gray')
    plt.title('Cover Noise Residual')
    plt.axis('off')
    
    plt.subplot(2, 4, 6)
    plt.imshow(result['stego_residual'], cmap='gray')
    plt.title('Stego Noise Residual')
    plt.axis('off')
    
    plt.subplot(2, 4, 7)
    plt.imshow(result['residual_diff'], cmap='hot')
    plt.title('Residual Difference')
    plt.axis('off')
    
    plt.subplot(2, 4, 8)
    plt.imshow(result['residual_diff_amplified'], cmap='hot')
    plt.title('Residual Difference (Amplified 10x)')
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

ModuleNotFoundError: No module named 'numpy'